In [45]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from http.server import HTTPServer, BaseHTTPRequestHandler
import threading
import webbrowser

In [46]:
SPOTIFY_CLIENT_ID = "e2997e27ae344ad98c9f4fde70a14191"         # Ganti dengan Client ID Spotify kamu
SPOTIFY_CLIENT_SECRET = "df36bf6e4bd0491698c1ad37ad131e82" # Ganti dengan Client Secret Spotify kamu
SPOTIFY_REDIRECT_URI = "http://localhost:5300/callback"
SPOTIFY_SCOPES = "user-read-playback-state user-modify-playback-state user-read-currently-playing playlist-read-private playlist-read-collaborative playlist-modify-private playlist-modify-public user-read-playback-position user-top-read user-read-recently-played user-read-email"

In [47]:
# Inisialisasi Spotipy OAuth
sp_oauth = SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri=SPOTIFY_REDIRECT_URI,
    scope=SPOTIFY_SCOPES
)

In [48]:
# Server HTTP untuk menangani callback
class SpotifyAuthHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        # Parsing URL untuk mendapatkan kode otorisasi
        if "/callback" in self.path:
            query = self.path.split("?")[-1]
            params = dict(qc.split("=") for qc in query.split("&"))
            code = params.get("code", None)

            if code:
                # Dapatkan token akses
                token_info = sp_oauth.get_access_token(code)
                access_token = token_info['access_token']

                # Inisialisasi Spotipy dengan token
                sp = spotipy.Spotify(auth=access_token)

                # Contoh penggunaan beberapa endpoint:
                # 1. Mendapatkan informasi pengguna
                user_data = sp.current_user()
                user_name = user_data['display_name']
                user_email = user_data['email']

                # 2. Mendapatkan lagu terakhir yang diputar
                recently_played = sp.current_user_recently_played(limit=5)
                recently_played_tracks = [
                    f"{item['track']['name']} - {item['track']['artists'][0]['name']}" 
                    for item in recently_played['items']
                ]

                # Cetak hasil di terminal
                print("\n=== User Info ===")
                print(f"Name: {user_name}")
                print(f"Email: {user_email}")
                print("\n=== Recently Played Tracks ===")
                for track in recently_played_tracks:
                    print(f"- {track}")

                # Balas ke browser
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b"Authentication successful! You can close this tab.")
                return

        # Jika tidak sesuai callback
        self.send_response(404)
        self.end_headers()
        self.wfile.write(b"Not Found")

# Menjalankan server lokal
def run_server():
    server = HTTPServer(("localhost", 5300), SpotifyAuthHandler)
    print("Starting server at http://localhost:5300...")
    server.serve_forever()

# Membuka URL autentikasi di browser
def open_auth_url():
    auth_url = sp_oauth.get_authorize_url()
    print("Opening the following URL for authentication:")
    print(auth_url)
    webbrowser.open(auth_url)

In [49]:
if __name__ == "__main__":
    # Menjalankan server di thread terpisah
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()

    # Membuka URL autentikasi
    open_auth_url()

    # Menunggu pengguna menyelesaikan autentikasi
    input("Press Enter to exit...\n")

Opening the following URL for authentication:
https://accounts.spotify.com/authorize?client_id=e2997e27ae344ad98c9f4fde70a14191&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A5300%2Fcallback&scope=user-read-playback-state+user-modify-playback-state+user-read-currently-playing+playlist-read-private+playlist-read-collaborative+playlist-modify-private+playlist-modify-public+user-read-playback-position+user-top-read+user-read-recently-played+user-read-email
Starting server at http://localhost:5300...


In [50]:
def get_spotify_instance():
    """Fungsi untuk mendapatkan instance Spotipy yang sudah diotorisasi."""
    auth_url = sp_oauth.get_authorize_url()
    print("Buka URL ini di browser untuk login Spotify:")
    print(auth_url)

    # Masukkan kode dari URL redirect
    response_url = input("Masukkan URL setelah login: ").strip()
    code = sp_oauth.parse_response_code(response_url)
    token_info = sp_oauth.get_access_token(code)
    access_token = token_info['access_token']

    # Return Spotify instance
    return spotipy.Spotify(auth=access_token)

def get_recently_played(sp):
    """Fungsi untuk mendapatkan lagu-lagu terakhir yang diputar."""
    results = sp.current_user_recently_played(limit=50)
    tracks = [
        f"{item['track']['name']} by {item['track']['artists'][0]['name']}" 
        for item in results['items']
    ]
    print("\n=== Recently Played Tracks ===")
    for track in tracks:
        print(f"- {track}")

# Main
if __name__ == "__main__":
    # Dapatkan Spotify instance
    sp = get_spotify_instance()

Buka URL ini di browser untuk login Spotify:
https://accounts.spotify.com/authorize?client_id=e2997e27ae344ad98c9f4fde70a14191&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A5300%2Fcallback&scope=user-read-playback-state+user-modify-playback-state+user-read-currently-playing+playlist-read-private+playlist-read-collaborative+playlist-modify-private+playlist-modify-public+user-read-playback-position+user-top-read+user-read-recently-played+user-read-email


C:\Users\Sanju\AppData\Local\Temp\ipykernel_18136\1400587458.py:10: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


In [51]:
def get_currently_playing(sp):
    """Fungsi untuk mendapatkan lagu yang sedang diputar."""
    current_track = sp.currently_playing()
    if current_track and current_track.get("is_playing"):
        track_name = current_track["item"]["name"]
        artist_name = current_track["item"]["artists"][0]["name"]
        print("\n=== Currently Playing ===")
        print(f"{track_name} by {artist_name}")
    else:
        print("\n=== Currently Playing ===")
        print("Tidak ada lagu yang sedang diputar.")


In [53]:
# Jalankan fungsi untuk mendapatkan lagu terakhir yang diputar
get_recently_played(sp)

# Jalankan fungsi untuk mendapatkan lagu yang sedang diputar
get_currently_playing(sp)



=== Recently Played Tracks ===
- Walk Em Down (feat. Roddy Ricch) by NLE Choppa
- SLUT ME OUT 2 by NLE Choppa
- SLUT ME OUT by NLE Choppa
- Ransom by Lil Tecca
- Rock by Stepz
- Embrace It by Ndotz
- Big Dawgs by Hanumankind
- Big Dawgs by Hanumankind
- Big Dawgs by Hanumankind
- Runtuh by Feby Putri
- Sialan by Adrian Khalif
- Lantas by Juicy Luicy
- Satu Bulan by Bernadya
- Kata Mereka Ini Berlebihan by Bernadya
- Ernie Zakri - Buai Laju-Laju (The Golden Swing) by Upin & Ipin
- Masih Disini by Bunkface
- Bangkit Anak Muda (BoBoiBoy) by Nizam Razak
- TAPOPS (BoBoiBoy Galaxy) by Anas Abdul Aziz

=== Currently Playing ===
Camelot by NLE Choppa


In [54]:
def create_mood_playlist_from_tracks(sp, user_id, mood, top_tracks):
    """
    Membuat playlist di akun pengguna berdasarkan lagu-lagu teratas dari playlist mood.
    """
    if not top_tracks:
        print(f"Tidak ada lagu yang ditemukan untuk mood: {mood}")
        return None

    playlist_name = f"Mood Playlist: {mood.capitalize()}"
    description = f"Playlist ini dibuat berdasarkan mood Anda: {mood.capitalize()}."

    # Membuat playlist di akun pengguna
    playlist = sp.user_playlist_create(
        user=user_id,
        name=playlist_name,
        public=True,
        description=description
    )
    playlist_id = playlist['id']
    print(f"Playlist berhasil dibuat: {playlist_name} (ID: {playlist_id})")

    # Menambahkan lagu ke playlist
    track_uris = [track['uri'] for track in top_tracks]
    sp.playlist_add_items(playlist_id, track_uris)
    print(f"Lagu telah ditambahkan ke playlist: {playlist_name}")

    return playlist_id


def get_top_tracks_from_playlist(sp, playlist_id):
    """
    Mendapatkan lagu populer dari playlist berdasarkan popularitas.
    """
    playlist_tracks = sp.playlist_tracks(playlist_id, limit=50)
    # Mengurutkan lagu berdasarkan popularitas (track popularity)
    sorted_tracks = sorted(playlist_tracks['items'], key=lambda x: x['track']['popularity'], reverse=True)

    top_tracks = []
    for track in sorted_tracks[:5]:  # Ambil 5 lagu terpopuler
        track_uri = track['track']['uri']
        track_name = track['track']['name']
        artist_name = track['track']['artists'][0]['name']
        track_popularity = track['track']['popularity']
        top_tracks.append({
            'uri': track_uri,
            'track_name': track_name,
            'artist_name': artist_name,
            'popularity': track_popularity
        })
        print(f" - {track_name} by {artist_name} (Popularity: {track_popularity})")

    return top_tracks


def get_playlist_for_mood(sp, mood_keywords):
    """
    Mencari playlist berdasarkan keyword mood dan mengambil 5 lagu terpopuler.
    """
    # Mencari playlist berdasarkan keyword mood
    playlist_results = sp.search(q=mood_keywords, type="playlist", limit=1)
    
    if playlist_results['playlists']['items']:
        playlist = playlist_results['playlists']['items'][0]
        print(f"\nPlaylist: {playlist['name']}")
        playlist_id = playlist['id']
        return get_top_tracks_from_playlist(sp, playlist_id)
    else:
        print("Playlist tidak ditemukan untuk mood:", mood_keywords)
        return []


def generate_mood_playlists(sp, user_id):
    """
    Membuat playlist untuk setiap mood berdasarkan lagu populer dari playlist terkait mood.
    """
    moods = {
        "happy": "happy",
        "sad": "sad",
        "neutral": "neutral",
        "surprised": "surprised",
        "angry": "angry"
    }

    for mood, keyword in moods.items():
        print(f"\n=== {mood.capitalize()} Mood ===")
        top_tracks = get_playlist_for_mood(sp, keyword)
        if top_tracks:
            create_mood_playlist_from_tracks(sp, user_id, mood, top_tracks)


# Eksekusi fungsi utama
user_id = sp.me()['id']  # Mendapatkan user ID pengguna Spotify
generate_mood_playlists(sp, user_id)


=== Happy Mood ===

Playlist: Pkoknya bisa naikin mood aja
 - Late Night Talking by Harry Styles (Popularity: 76)
 - No Other Heart by Mac DeMarco (Popularity: 74)
 - NVMD by Denise Julia (Popularity: 72)
 - Satellite by Harry Styles (Popularity: 72)
 - Make You Mine by PUBLIC (Popularity: 70)
Playlist berhasil dibuat: Mood Playlist: Happy (ID: 1xzbgq8nw3en8rQS4NYjR4)
Lagu telah ditambahkan ke playlist: Mood Playlist: Happy

=== Sad Mood ===

Playlist: Sad song' it's time to cry  ಥ_ಥ
 - Yellow by Coldplay (Popularity: 89)
 - Another Love by Tom Odell (Popularity: 86)
 - Perfect by Ed Sheeran (Popularity: 86)
 - Photograph by Ed Sheeran (Popularity: 84)
 - Somewhere Only We Know by Keane (Popularity: 83)
Playlist berhasil dibuat: Mood Playlist: Sad (ID: 6Ht4VU8Awd6JAWXDKEbRuE)
Lagu telah ditambahkan ke playlist: Mood Playlist: Sad

=== Neutral Mood ===

Playlist: she probably thinks neutral milk hotel is underground
 - There Is a Light That Never Goes Out - 2011 Remaster by The Smiths 